# *NOTE* : Sample of dataset used due to huge size of original dataset resulting in memory error.


In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

import logging

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

import gc

In [ ]:
from google.colab import drive
drive.mount('/content/grive')

Mounted at /content/grive


In [ ]:
!pwd

/content


In [ ]:
cd /content/grive/MyDrive/Colab Notebooks/NLP Data/

/content/grive/MyDrive/Colab Notebooks/NLP Data


In [ ]:
# Reading in the data

df = pd.read_csv('/content/grive/MyDrive/Colab Notebooks/NLP Data/blogs/blogtext.csv')

# Analysing Data

print(df.head())
print('\n')
print('-'*60)
print('\n')
df.info()

        id gender  age              topic      sign          date  \
0  2059027   male   15            Student       Leo   14,May,2004   
1  2059027   male   15            Student       Leo   13,May,2004   
2  2059027   male   15            Student       Leo   12,May,2004   
3  2059027   male   15            Student       Leo   12,May,2004   
4  3581210   male   33  InvestmentBanking  Aquarius  11,June,2004   

                                                text  
0             Info has been found (+/- 100 pages,...  
1             These are the team members:   Drewe...  
2             In het kader van kernfusie op aarde...  
3                   testing!!!  testing!!!            
4               Thanks to Yahoo!'s Toolbar I can ...  


------------------------------------------------------------


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681284 entries, 0 to 681283
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   i

In [ ]:
df.shape

(681284, 7)

In [ ]:
list(df.columns)
#df.columns

['id', 'gender', 'age', 'topic', 'sign', 'date', 'text']

In [ ]:
print(df.topic.value_counts())
print('\n')
print('-'*60)
print('\n')
print("Total number of unique topics : ", df.topic.unique().size)

indUnk                     251015
Student                    153903
Technology                  42055
Arts                        32449
Education                   29633
Communications-Media        20140
Internet                    16006
Non-Profit                  14700
Engineering                 11653
Law                          9040
Publishing                   7753
Science                      7269
Government                   6907
Consulting                   5862
Religion                     5235
Fashion                      4851
Marketing                    4769
Advertising                  4676
BusinessServices             4500
Banking                      4049
Chemicals                    3928
Telecommunications           3891
Accounting                   3832
Military                     3128
Museums-Libraries            3096
Sports-Recreation            3038
HumanResources               3010
RealEstate                   2870
Transportation               2326
Manufacturing 

In [ ]:
# Checking for null values

df.isna().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

In [ ]:
# Drop columns with no significance here

df = df.drop(['id','gender','age','sign','date'], axis = 1)

In [ ]:
#df.describe(include = [np.number]).T
#df.describe(include = [object]).T

In [ ]:
# Changing 'topic' column data type

catagorical_coloumn = ["topic"]

df[catagorical_coloumn] = df[catagorical_coloumn].astype("category")

In [ ]:
df.head()

,topic,text
0,Student,"Info has been found (+/- 100 pages,..."
1,Student,These are the team members: Drewe...
2,Student,In het kader van kernfusie op aarde...
3,Student,testing!!! testing!!!
4,InvestmentBanking,Thanks to Yahoo!'s Toolbar I can ...


Outcome Of Data Analysis
-------------------------
1. No null values in the Data set
2. Total number of unique topics :  40
3. There are 6 different columns - 'id', 'gender', 'age', 'topic', 'sign', 'date', 'text'
4. Columns to be dropped as they dont hold significance in our current classification - id, gender, age, sign, date

In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.1 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=b429e48f126f71165a9c7e95f776c04c006821ef8bb8ff780f309d5155ed3263
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [ ]:
# Using langdetect to detect language for each review

from langdetect import detect

def detect_english(text):
  try:
    return detect(text) == 'en'
  except:
    return False

In [ ]:
df_english = []
for i in df.index:
  df_english.append(detect_english(df['text'].iloc[i]))

# Count of values with df_english array

pd.Series(df_english).value_counts()

True     651448
False     29836
dtype: int64

In [ ]:
df['df_english'] = pd.Series(df_english)
df.head()

,topic,text,df_english
0,Student,"Info has been found (+/- 100 pages,...",True
1,Student,These are the team members: Drewe...,False
2,Student,In het kader van kernfusie op aarde...,True
3,Student,testing!!! testing!!!,True
4,InvestmentBanking,Thanks to Yahoo!'s Toolbar I can ...,True


In [ ]:
# Filtering the reviews detected as english by langdetect

df_english = pd.DataFrame()
df_english = df.loc[df['df_english'] == True]

df_english = df_english.drop(['df_english'], axis = 1)
df_english.shape

(651448, 2)

In [ ]:
df_english.dtypes

topic    category
text       object
dtype: object

In [ ]:
log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())

In [ ]:
# Text data pre-processing and cleanup

def remove_special_characters(df):
  log.info("Removing special characters from dataframe")
  no_special_characters = df.replace(r'[^A-Za-z0-9]+',' ', regex=True)
  return no_special_characters

def remove_numbers(df):
  log.info("Removing numbers from dataframe")
  df = pd.Series(df.values.flatten())
  removed_numbers = df.str.replace(r'\d+',' ')
  return removed_numbers

In [ ]:
df_english.to_csv('df_text_file.csv', index = False) # encoding = 'utf-8-sig'
# files.download('df_text_file.csv')


In [ ]:
df_english = pd.read_csv('/content/grive/MyDrive/Colab Notebooks/NLP Data/df_text_file.csv')

In [ ]:
df_english['text']= remove_special_characters(df_english['text'])
df_english['text'] = remove_numbers(df_english['text'])

<ipython-input-21-605dd92ec6ff>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  removed_numbers = df.str.replace(r'\d+',' ')


In [ ]:
df_english.memory_usage().sum()

10423296

In [ ]:
# TAKING A 10% SUBSET OF THE DATASET DUE TO ITS HUGE SIZE THAT RESULTS IN MEMORY CRASH

df_english_1 = df_english.sample(frac =.10)
df_english_1.head(100)

,topic,text
569823,Non-Profit,Random Quote bad News For Tai The Bigger They...
101782,indUnk,Yeay nbsp Anyway what in the hell is the S D ...
594388,indUnk,MAYDAY MAYDAY MAYDAY Those are pearls that we...
352254,Student,Too many people associate tarot with fortune ...
413318,Banking,As I ve mentioned before I work in a call cen...
...,...,...
77897,Arts,I have had a lazy lazy day Boy and Girl have ...
183455,Communications-Media,PM to hockey clean up your act In what I thou...
56013,indUnk,Leighton and Young created an imaginary Korea...
556036,Communications-Media,urlLink Yahoo News D Church Opened to Woo In...


In [ ]:
df_english_1.memory_usage().sum()

1563480

In [ ]:
df_english_1.shape

(65145, 2)

In [ ]:
# Checking for null values

df_english_1.isna().sum()

topic    0
text     0
dtype: int64

In [ ]:
# Text pre-processing

def lowercase(df):
  #df.astype('str')
  logging.info("Converting dataframe to lowercase")
  lowercase_df = df.apply(lambda x: x.lower())
  return lowercase_df

In [ ]:
df_english_1['text'] = lowercase(df_english_1['text'])
df_english_1.head()

,topic,text
569823,Non-Profit,random quote bad news for tai the bigger they...
101782,indUnk,yeay nbsp anyway what in the hell is the s d ...
594388,indUnk,mayday mayday mayday those are pearls that we...
352254,Student,too many people associate tarot with fortune ...
413318,Banking,as i ve mentioned before i work in a call cen...


In [ ]:
.nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Remove stop words from reviews

stops = set(stopwords.words('english'))

df_english_1['text'] = df_english_1['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stops)]))
df_english_1.head()

,topic,text
569823,Non-Profit,random quote bad news tai bigger farther fall ...
101782,indUnk,yeay nbsp anyway hell nbsp come erik nbsp bugg...
594388,indUnk,mayday mayday mayday pearls eyes nothing doth ...
352254,Student,many people associate tarot fortune telling si...
413318,Banking,mentioned work call center bank take escalated...


In [ ]:
def remove_whitespace(df):
  log.info("Removing extra whitespace from dataframe")
  # replace more than 1 space with 1 space
  merged_spaces = pd.Series(df).str.replace(r"\s\s+",' ')
  # delete beginning and trailing spaces
  trimmed_spaces = pd.DataFrame(merged_spaces).apply(lambda x: x.strip() if isinstance(x, str) else x)
  return trimmed_spaces

In [ ]:
df_english_1['text'] = remove_whitespace(df_english_1['text'])

<ipython-input-34-10cabc4b9871>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_spaces = pd.Series(df).str.replace(r"\s\s+",' ')


In [ ]:
type(df_english_1['text'])

pandas.core.series.Series

In [ ]:
gc.collect()
df_english_1.head()

,topic,text
569823,Non-Profit,random quote bad news tai bigger farther fall ...
101782,indUnk,yeay nbsp anyway hell nbsp come erik nbsp bugg...
594388,indUnk,mayday mayday mayday pearls eyes nothing doth ...
352254,Student,many people associate tarot fortune telling si...
413318,Banking,mentioned work call center bank take escalated...


In [ ]:
df_english_1.to_csv('df_english_file.csv', index = False, encoding = 'utf-8-sig', chunksize=10000)

In [ ]:
df_english_1 = pd.read_csv('/content/grive/MyDrive/Colab Notebooks/NLP Data/df_english_file.csv')

In [ ]:
# Checking for any new null values

df_english_1.isna().sum()

topic     0
text     17
dtype: int64

In [ ]:
# Dropping The Null Values

df_english_1 = df_english_1.dropna()
df_english_1.isna().sum()

topic    0
text     0
dtype: int64

In [ ]:
# Independent & Target variable creation

from sklearn.model_selection import train_test_split

X = df_english_1.text
y = df_english_1.topic

In [ ]:
# Target variable values

y.value_counts()

indUnk                     24057
Student                    14860
Technology                  3968
Arts                        3153
Education                   2852
Communications-Media        1920
Internet                    1526
Non-Profit                  1329
Engineering                 1048
Law                          861
Publishing                   740
Science                      706
Government                   671
Consulting                   579
Marketing                    502
Religion                     472
Advertising                  440
Fashion                      416
BusinessServices             395
Chemicals                    388
Banking                      388
Accounting                   368
Telecommunications           354
HumanResources               313
Sports-Recreation            299
Military                     293
Museums-Libraries            278
RealEstate                   245
Biotech                      231
Manufacturing                218
Transporta

Inference From y value counts
--------------------------

y is multiclass with high data imbalance

# **Label Encoding the y Variable**

In [ ]:
# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in categorical columns
y_encoded = label_encoder.fit_transform(y)

In [ ]:
#y_encoded = pd.get_dummies(y)

In [ ]:
type(X), X.shape, type(y_encoded), y_encoded.shape

(pandas.core.series.Series, (65128,), numpy.ndarray, (65128,))

# **Performing the Train & Test Split**

In [ ]:
X_train, X_test, y_train, y_test =train_test_split(X, y_encoded, random_state=42,
                                                   test_size = 0.1,
                                                  shuffle = True)

In [ ]:
type(X_train), type(y_train)

(pandas.core.series.Series, numpy.ndarray)

In [ ]:
y_train

array([34,  6, 39, ..., 17,  6, 10])

In [ ]:
print("shape of training set :", X_train.shape)
print("shape of test set :", X_test.shape)

shape of training set : (58615,)
shape of test set : (6513,)


# **Feature Extraction Using TFIDF Vectorization (VECTORIZER 1)**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train_copy = X_train
y_train_copy = y_train

X_test_copy = X_test
y_test_copy = y_test

stops = set(stopwords.words('english'))
# tfidf = TfidfVectorizer(stop_words=stop_words, tokenizer=tokenize, vocabulary=vocabulary)

vectorizer = TfidfVectorizer(stop_words = stops)
vectorizer.fit(X_train_copy)

X_train_copy = vectorizer.transform(X_train_copy)
X_test_copy = vectorizer.transform(X_test_copy)
X_train_copy, X_test_copy

(<58615x153610 sparse matrix of type '<class 'numpy.float64'>'
 	with 4607809 stored elements in Compressed Sparse Row format>,
 <6513x153610 sparse matrix of type '<class 'numpy.float64'>'
 	with 512201 stored elements in Compressed Sparse Row format>)

In [ ]:
X_train_copy

<58615x153610 sparse matrix of type '<class 'numpy.float64'>'
	with 4607809 stored elements in Compressed Sparse Row format>

In [ ]:
X_train_copy.shape, type(X_train_copy)

((58615, 153610), scipy.sparse.csr.csr_matrix)

In [ ]:
# MultinomialNB Model creation, fitting on Train data and performance analysis

from sklearn.metrics import classification_report

def performance_metrics(y_test,y_predict):
  print(classification_report(y_test,y_predict))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

model_mnb = MultinomialNB()

model_mnb.fit(X_train_copy, y_train_copy)
y_predict_copy = model_mnb.predict(X_test_copy)

print('Multinomial Naive Bayes Model Accuracy : ',accuracy_score(y_test_copy, y_predict_copy))
print('\n')

performance_metrics(y_test_copy, y_predict_copy)

Multinomial Naive Bayes Model Accuracy :  0.3809304467987103


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.00      0.00      0.00        59
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00       304
           5       0.00      0.00      0.00         9
           6       0.00      0.00      0.00        32
           7       0.00      0.00      0.00        24
           8       0.00      0.00      0.00        38
           9       0.00      0.00      0.00        43
          10       0.00      0.00      0.00       185
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00        64
          13       0.00      0.00      0.00       283
          14       0.00      0.00      0.00       101
          15       0.00      0.00      0.00         3
          16      

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gensim
from gensim.models import Word2Vec

X_train_arr = X_train.to_numpy()
X_test_arr = X_test.to_numpy()

In [ ]:
type(X_train_arr), X_train_arr.shape

(numpy.ndarray, (58615,))

# **Skip gram Vectorization Of Train Data Using Gensim Word2Vec (VECTORIZER 2)**

In [ ]:
Word2Vec_model_Xtrained_skipgram = Word2Vec(X_train_arr, size = 150, workers = 4, sg = 1)
X_train_wv_skipgram = X_train.apply(lambda text: np.mean([Word2Vec_model_Xtrained_skipgram.wv[word] for word in text.split() if word in Word2Vec_model_Xtrained_skipgram.wv]))

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
type(X_train_wv_skipgram), X_train_wv_skipgram.shape

(pandas.core.series.Series, (58615,))

In [ ]:
type(y_train), y_train.shape

(numpy.ndarray, (58615,))

In [ ]:
np.unique(y_train).sum()

780

In [ ]:
y_train = pd.Series(y_train)
#type(y_train)
train_skipgram_data = pd.concat([X_train_wv_skipgram.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train_skipgram_data.head()

,text,0
0,-0.00546,34
1,NaN,6
2,NaN,39
3,NaN,39
4,NaN,39


In [ ]:
train_skipgram_data.shape
print('-'*60)
print('Null Values Post Skip Gram Vectorization in train data:\n', train_skipgram_data.isna().sum())
train_skipgram_data = train_skipgram_data.dropna()
print('-'*60)
print('Shape of Vectorized Train Set post removing Null Values :',train_skipgram_data.shape)

------------------------------------------------------------
Null Values Post Skip Gram Vectorization in train data:
 text    43992
0           0
dtype: int64
------------------------------------------------------------
Shape of Vectorized Train Set post removing Null Values : (14623, 2)


In [ ]:
train_skipgram_data = train_skipgram_data.drop_duplicates()
train_skipgram_data.shape

(5042, 2)

# **Skip gram Vectorization Of Test Data Using Gensim Word2Vec (VECTORIZER 3)**

In [ ]:
Word2Vec_model_Xtest_skipgram = Word2Vec(X_test_arr, size = 150, workers = 4, sg = 1)
X_test_wv_skipgram = X_test.apply(lambda text: np.mean([Word2Vec_model_Xtest_skipgram.wv[word] for word in text.split() if word in Word2Vec_model_Xtest_skipgram.wv]))

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
print(type(X_test_wv_skipgram))
print('-'*60)
y_test = pd.Series(y_test)
#type(y_train)
test_skipgram_data = pd.concat([X_test_wv_skipgram.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
test_skipgram_data.head()

<class 'pandas.core.series.Series'>
------------------------------------------------------------


,text,0
0,NaN,10
1,0.000698,34
2,NaN,39
3,-0.000776,39
4,NaN,39


In [ ]:
print('Null Values Post Skip Gram Vectorization in Test data:', test_skipgram_data.isna().sum())

test_skipgram_data = test_skipgram_data.dropna()
print('-'*60)
print('Shape of Vectorized Test Set Post removing Null Values : ',test_skipgram_data.shape)

Null Values Post Skip Gram Vectorization in Test data: text    4890
0          0
dtype: int64
------------------------------------------------------------
Shape of Vectorized Test Set Post removing Null Values :  (1623, 2)


In [ ]:
test_skipgram_data = test_skipgram_data.drop_duplicates()
test_skipgram_data.shape

(838, 2)

# Saving the Skip Gram Vectorised Train and Test Data and Models

In [ ]:
train_skipgram_data.to_csv('train_skipgram_data.csv', index = False, encoding = 'utf-8-sig', chunksize=5000)
test_skipgram_data.to_csv('test_skipgram_data.csv', index = False, encoding = 'utf-8-sig', chunksize=5000)

Word2Vec_model_Xtrained_skipgram.save('/content/grive/MyDrive/Colab Notebooks/NLP Data/Word2Vec_model_Xtrain_skipgram_file.model')
Word2Vec_model_Xtest_skipgram. save('/content/grive/MyDrive/Colab Notebooks/NLP Data/Word2Vec_model_Xtest_skipgram_file.model')

train_skipgram_data = pd.read_csv('/content/grive/MyDrive/Colab Notebooks/NLP Data/train_skipgram_data.csv')
test_skipgram_data = pd.read_csv('/content/grive/MyDrive/Colab Notebooks/NLP Data/test_skipgram_data.csv')



# **CBOW Vectorization Of Train and Test Data Using Gensim Word2Vec**

In [ ]:
Word2Vec_model_Xtest_cbow = Word2Vec(X_test_arr, size = 150, workers = 4, sg = 0)
X_test_wv_cbow = X_test.apply(lambda text: np.mean([Word2Vec_model_Xtest_cbow.wv[word] for word in text.split() if word in Word2Vec_model_Xtest_cbow.wv]))

Word2Vec_model_Xtrain_cbow = Word2Vec(X_train_arr, size = 150, workers = 4, sg = 0)
X_train_wv_cbow = X_train.apply(lambda text: np.mean([Word2Vec_model_Xtrain_cbow.wv[word] for word in text.split() if word in Word2Vec_model_Xtrain_cbow.wv]))

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
print(type(X_train_wv_cbow))
print('-'*60)
#y_test = pd.Series(y_test)
#type(y_train)
train_cbow_data = pd.concat([X_train_wv_cbow.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train_cbow_data.head()

<class 'pandas.core.series.Series'>
------------------------------------------------------------


,text,0
0,-0.015387,34
1,NaN,6
2,NaN,39
3,NaN,39
4,NaN,39


In [ ]:
print(type(X_test_wv_cbow))
print('-'*60)
#y_train = pd.Series(y_train)
#type(y_train)
test_cbow_data = pd.concat([X_test_wv_cbow.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
test_cbow_data.head()

<class 'pandas.core.series.Series'>
------------------------------------------------------------


,text,0
0,NaN,10
1,0.005809,34
2,NaN,39
3,0.004355,39
4,NaN,39


In [ ]:
print('Null Values Post Continuous Bag Of Words Vectorization in Train data:', train_cbow_data.isna().sum())
train_cbow_data = train_cbow_data.dropna()
print('Shape of Vectorized Train Data post removal of null values', train_cbow_data.shape)

Null Values Post Continuous Bag Of Words Vectorization in Train data: text    43992
0           0
dtype: int64
Shape of Vectorized Train Data post removal of null values (14623, 2)


In [ ]:
print('Null Values Post Continuous Bag Of Words Vectorization in Test data:', test_cbow_data.isna().sum())
test_cbow_data = test_cbow_data.dropna()
print('Shape of Vectorized Test Data post removal of null values', test_cbow_data.shape)

Null Values Post Continuous Bag Of Words Vectorization in Test data: text    4890
0          0
dtype: int64
Shape of Vectorized Test Data post removal of null values (1623, 2)


In [ ]:
train_cbow_data = train_cbow_data.drop_duplicates()
train_cbow_data.shape

(4973, 2)

In [ ]:
test_cbow_data = test_cbow_data.drop_duplicates()
test_cbow_data.shape

(829, 2)

# Saving the CBOW Vectorised Train and Test Data and Models

In [ ]:
train_cbow_data.to_csv('train_cbow_data_file.csv', index = False, encoding = 'utf-8-sig', chunksize=5000)
test_cbow_data.to_csv('test_cbow_data_file.csv', index = False, encoding = 'utf-8-sig', chunksize=5000)

Word2Vec_model_Xtrain_cbow.save('/content/grive/MyDrive/Colab Notebooks/NLP Data/Word2Vec_model_Xtrain_cbow_file.model')
Word2Vec_model_Xtest_cbow.save('/content/grive/MyDrive/Colab Notebooks/NLP Data/Word2Vec_model_Xtest_cbow_file.model')

train_cbow_data = pd.read_csv('/content/grive/MyDrive/Colab Notebooks/NLP Data/train_cbow_data_file.csv')
test_cbow_data = pd.read_csv('/content/grive/MyDrive/Colab Notebooks/NLP Data/test_cbow_data_file.csv')



# **Text Classification of Skipgram Vectorized Dataset Using Machine Learning Models**

In [ ]:
X_test_wv_skipgram = test_skipgram_data.text
y_test = test_skipgram_data.iloc[:,1]
X_test_wv_skipgram.shape, y_test.shape

((838,), (838,))

In [ ]:
X_train_wv_skipgram = train_skipgram_data.text
y_train = train_skipgram_data.iloc[:,1]
X_train_wv_skipgram.shape, y_train.shape

((5042,), (5042,))

In [ ]:
type(X_train_wv_skipgram), type(y_train)

(pandas.core.series.Series, pandas.core.series.Series)

# Converting the Train and Test Datasets to Dataframes

In [ ]:
X_train_wv_skipgram = pd.DataFrame(X_train_wv_skipgram)
y_train = pd.DataFrame(y_train)
X_train_wv_skipgram.shape, y_train.shape

((5042, 1), (5042, 1))

In [ ]:
X_test_wv_skipgram = pd.DataFrame(X_test_wv_skipgram)
y_test = pd.DataFrame(y_test)
X_test_wv_skipgram.shape, y_test.shape

((838, 1), (838, 1))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Fit the model on train
model_base = LogisticRegression()
model_base.fit(X_train_wv_skipgram, y_train)
#predict on test
Y_predict = model_base.predict(X_test_wv_skipgram)
print("Accuracy of Logistic Regression Base Model on Test Data without Hyperparameter Tuning = ", model_base.score(X_test_wv_skipgram, y_test))

print('-'*60)

NNH_base = KNeighborsClassifier()
NNH_base.fit(X_train_wv_skipgram, y_train)
Y_predict = NNH_base.predict(X_test_wv_skipgram)
print("Accuracy of KNN Base Model on Test Data without Hyperparameter Tuning = ", NNH_base.score(X_test_wv_skipgram, y_test))

print('-'*60)

svc_base = SVC()
svc_base.fit(X_train_wv_skipgram, y_train)
Y_predict = svc_base.predict(X_test_wv_skipgram)
print("Accuracy of SVC Base Model on Test Data without Hyperparameter Tuning =", svc_base.score(X_test_wv_skipgram, y_test))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of Logistic Regression Base Model on Test Data without Hyperparameter Tuning =  0.2386634844868735
------------------------------------------------------------
Accuracy of KNN Base Model on Test Data without Hyperparameter Tuning =  0.18973747016706444
------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of SVC Base Model on Test Data without Hyperparameter Tuning = 0.2386634844868735


# **ML Models Hyperparameter Tuning for Skipgram Vectorized Dataset**

# Hyperparameter Tuning For KNN Model

In [ ]:
# Hyperparameter Tuning for KNN Model for Skipgram Vectorized Model

from sklearn.model_selection import GridSearchCV

neighbors = range(1,100)
grid_params = { 'n_neighbors' : neighbors,
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}

gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)

# fit the Grid Search model for KNN model on our train set
g_res = gs.fit(X_train_wv_skipgram, y_train)

Fitting 3 folds for each of 594 candidates, totalling 1782 fits


/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
# get the hyperparameters with the above best score
print('Hyperparameters for the KNN Model :', g_res.best_params_)

# find the best score for our KNN model on Train Data after Hyperparameter tuning
print("Best mean accuracy of KNN model on Train Data with Hyperparameter Tuning done = ", g_res.best_score_)

# Accuracy of Hyperparameter tuned KNN model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned KNN Model =", g_res.score(X_test_wv_skipgram, y_test))

Hyperparameters for the KNN Model : {'metric': 'minkowski', 'n_neighbors': 74, 'weights': 'uniform'}
Best mean accuracy of KNN model on Train Data with Hyperparameter Tuning done =  0.2600165953750129
Accuracy for the Test Data with Hyperparameter Tuned KNN Model = 0.2315035799522673


In [ ]:
y_predict_skipgram = g_res.predict(X_test_wv_skipgram)
performance_metrics(y_test, y_predict_skipgram)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        11
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        41
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00        34
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00        10
          13       0.00      0.00      0.00        46
          14       0.00      0.00      0.00        23
          15       0.00      0.00      0.00         2
          16       0.00      0.00      0.00         7
          17       0.00      0.00      0.00         8
          18       0.00    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Hyperparameter Tuning For SVM Model

In [ ]:
# Hyperparameter Tuning for SVM Model for Skipgram Vectorized dataset

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','rbf']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
best_grid = grid.fit(X_train_wv_skipgram, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.266 total time=   0.8s


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.265 total time=   0.8s


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 4/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.265 total time=   1.3s


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.265 total time=   1.3s


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# print best parameters after Hyperparameter tuning
print('Hyperparameters for the SVM Model :', best_grid.best_params_)

# find the best score for our SVM model on Train Data after Hyperparameter tuning
print("Best cross-validated mean accuracy of SVM model with Hyperparameter Tuning done = ", best_grid.best_score_)

# Accuracy of Hyperparameter tuned SVM model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned SVM Model =", best_grid.score(X_test_wv_skipgram, y_test))

Hyperparameters for the SVM Model : {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
Best cross-validated mean accuracy of SVM model with Hyperparameter Tuning done =  0.26497416112133654
Accuracy for the Test Data with Hyperparameter Tuned SVM Model = 0.2386634844868735


In [ ]:
y_predict_skipgram = best_grid.predict(X_test_wv_skipgram)
performance_metrics(y_test, y_predict_skipgram)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        11
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        41
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00        34
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00        10
          13       0.00      0.00      0.00        46
          14       0.00      0.00      0.00        23
          15       0.00      0.00      0.00         2
          16       0.00      0.00      0.00         7
          17       0.00      0.00      0.00         8
          18       0.00    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **ML Models Hyperparameter Tuning for CBOW Vectorized dataset**

In [ ]:
X_test_wv_cbow = test_cbow_data.text
y_test = test_cbow_data.iloc[:,1]
X_test_wv_cbow.shape, y_test.shape

((829,), (829,))

In [ ]:
X_train_wv_cbow = train_cbow_data.text
y_train = train_cbow_data.iloc[:,1]
X_train_wv_cbow.shape, y_train.shape

((4973,), (4973,))

In [ ]:
type(X_train_wv_cbow), type(y_train)

(pandas.core.series.Series, pandas.core.series.Series)

# Converting the Train and Test Datasets to Dataframes

In [ ]:
X_train_wv_cbow = pd.DataFrame(X_train_wv_cbow)
y_train = pd.DataFrame(y_train)
X_train_wv_cbow.shape, y_train.shape

((4973, 1), (4973, 1))

In [ ]:
X_test_wv_cbow = pd.DataFrame(X_test_wv_cbow)
y_test = pd.DataFrame(y_test)
X_test_wv_cbow.shape, y_test.shape

((829, 1), (829, 1))

# Hyperparameter Tuning For KNN Model

In [ ]:
# Hyperparameter Tuning for KNN Model for cbow Vectorized dataset

from sklearn.model_selection import GridSearchCV

neighbors = range(1,100)
grid_params = { 'n_neighbors' : neighbors,
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}

gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)

# fit the Grid Search model for KNN model on our train set
g_res = gs.fit(X_train_wv_cbow, y_train)

Fitting 3 folds for each of 594 candidates, totalling 1782 fits


/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
# get the hyperparameters with the above best score
print('Hyperparameters for the KNN Model :',g_res.best_params_)

# find the best score for our KNN model on Train Data after Hyperparameter tuning
print("Best mean accuracy of KNN model on Train Data with Hyperparameter Tuning done = ", g_res.best_score_)

# Accuracy of Hyperparameter tuned KNN model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned KNN Model =", g_res.score(X_test_wv_cbow, y_test))

Hyperparameters for the KNN Model : {'metric': 'minkowski', 'n_neighbors': 54, 'weights': 'uniform'}
Best mean accuracy of KNN model on Train Data with Hyperparameter Tuning done =  0.2658358163718688
Accuracy for the Test Data with Hyperparameter Tuned KNN Model = 0.24004825090470447


In [ ]:
y_predict_cbow = g_res.predict(X_test_wv_cbow)
performance_metrics(y_test, y_predict_cbow)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        37
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00        36
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00        10
          13       0.00      0.00      0.00        43
          14       0.00      0.00      0.00        23
          15       0.00      0.00      0.00         2
          16       0.00      0.00      0.00         7
          17       0.00      0.00      0.00         8
          18       0.00    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Hyperparameter Tuning For SVM Model

In [ ]:
# Hyperparameter Tuning for SVM Model for cbow Vectorized dataset

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','rbf']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
best_grid = grid.fit(X_train_wv_cbow, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.266 total time=   0.8s


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.267 total time=   0.8s


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 4/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.267 total time=   1.3s


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.267 total time=   1.3s


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# print best parameter after Hyperparameter tuning
print('Hyperparameters for the SVM Model :', best_grid.best_params_)

# find the best score for our SVM model on Train Data after Hyperparameter tuning
print("Best mean accuracy of SVM model on Train Data with Hyperparameter Tuning done = ", best_grid.best_score_)

# Accuracy of Hyperparameter tuned SVM model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned SVM Model =", best_grid.score(X_test_wv_cbow, y_test))

Hyperparameters for the SVM Model : {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
Best mean accuracy of SVM model on Train Data with Hyperparameter Tuning done =  0.26663983903420524
Accuracy for the Test Data with Hyperparameter Tuned SVM Model = 0.2436670687575392


In [ ]:
y_predict_cbow = best_grid.predict(X_test_wv_cbow)
performance_metrics(y_test, y_predict_cbow)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        37
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00        36
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00        10
          13       0.00      0.00      0.00        43
          14       0.00      0.00      0.00        23
          15       0.00      0.00      0.00         2
          16       0.00      0.00      0.00         7
          17       0.00      0.00      0.00         8
          18       0.00    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **ML Models Hyperparameter Tuning for TFIDF Vectorized dataset**

# Hyperparameter Tuning For Multinomial Naive Bayes Model

In [ ]:
param={'alpha': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000]}

gs_clf = GridSearchCV(MultinomialNB(),param,cv=10,return_train_score=True)
best_clf = gs_clf.fit(X_train_copy, y_train_copy)

In [ ]:
# print best parameter after Hyperparameter tuning
print('Hyperparameters for the Multinomial NB Model :',best_clf.best_params_)

# find the best score for our Multinomial NB model on Train Data after Hyperparameter tuning
print("Best mean accuracy of Multinomial NB model on Train Data with Hyperparameter Tuning done = ", best_clf.best_score_)

# Accuracy of Hyperparameter tuned Multinomial NB model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned Multinomial NB Model =", best_clf.score(X_test_copy, y_test_copy))


Hyperparameters for the Multinomial NB Model : {'alpha': 0.1}
Best mean accuracy of Multinomial NB model on Train Data with Hyperparameter Tuning done =  0.4037192311057408
Accuracy for the Test Data with Hyperparameter Tuned Multinomial NB Model = 0.4101028711807155


In [ ]:
y_predict_copy = best_clf.predict(X_test_copy)
performance_metrics(y_test_copy, y_predict_copy)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.00      0.00      0.00        59
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00        11
           4       0.75      0.01      0.02       304
           5       0.00      0.00      0.00         9
           6       0.00      0.00      0.00        32
           7       0.00      0.00      0.00        24
           8       0.00      0.00      0.00        38
           9       0.00      0.00      0.00        43
          10       0.00      0.00      0.00       185
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00        64
          13       0.00      0.00      0.00       283
          14       0.00      0.00      0.00       101
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00        46
          17       0.00    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5.B.
Multinomial NB Supervised ML algorithm gave the highest Train & Test Accuracy of 40.37 & 41 respectively.

Probable Reason : 
1. The Multinomial Naive Bayes can be accepted as the probabilistic approach to classifying documents in the case of acknowledging the frequency of a specified word in a text document.
2. simple and robust.
3. assigns documents to classes based on the statistical analysis of their contents.
4. relies on an assumption of conditional independence of features given a class, which is often a good first approximation to real-world phenomena.
5. they predict the tag of a text. They calculate the probability of each tag for a given text and then output the tag with the highest one.

5.A.
TFID performed better since it worked with Multinomial NB algorithm. Negative values in Word2Vec - Skipgram & CBOW vectorization were not compatible with Multinomial NB ML algorithm.

5.C.
Multinomial NB Hyperparameter alpha = 0.001 helped to improve the performance of the algorithm.
Probable Reason :
Algorithm's ability in classifying unseen words during training - for more complex sentences and by that I mean for sentences that may be more and more out of context relative to the training set, the noise will increase. In such cases increasing 'alpha' will be needed to smooth out the class separation. 

5.D. F1-Score to be given highest importance for multi-class classification since it gives a trade-off between Precision & Recall. Also the classes here have high data imbalance